In [51]:
# Imports
import sqlite3 as lite
import pandas as pd
import re
import numpy as np
import ast

In [52]:
conn = lite.connect('cycling_big.db')

riders_df = pd.read_sql_query('SELECT * FROM riders;', conn)
races_df = pd.read_sql_query('SELECT * FROM race_results', conn)

print("Amount of rows in races_df: ", races_df[races_df.columns[0]].count())
print("Amount of rows in riders_df: ", riders_df[riders_df.columns[0]].count())

conn.close()

"""
Convert the 'Date' column to DateTime format
Regarding races_df dataframe!
"""

races_df['Date'] = pd.to_datetime(races_df['Date'], errors='coerce', format='%d %B %Y')

# Remove rows where 'Date' is NaT
races_df = races_df.dropna(subset=['Date'])

# Normalize 'Date' to strip out time if it's present (this keeps just the date part)
races_df['Date'] = races_df['Date'].dt.normalize()

# Extract the month and year from the column and put them in their own columns
races_df['Month'] = races_df['Date'].dt.month
races_df['Year'] = races_df['Date'].dt.year

print("Unique values for 'months': ", races_df['Month'].unique(), "\n")
print("Unique values for 'years': ",races_df['Year'].unique())

"""
Converting the timetable to total seconds
"""

def time_to_seconds(time_str):
    # Remove commas and any spaces
    time_str = time_str.replace(',', '').strip()

    # Check the consistency of the time format using regular expressions
    match = re.match(r'(\d{1,2}):(\d{2}):(\d{2})', time_str)
    if match:
        hours, minutes, seconds = map(int, match.groups())
        total_seconds = hours * 3600 + minutes * 60 + seconds
        if total_seconds == 0:
            return np.nan
        return total_seconds

    # Do the same as the loop above, but now for MM:SS format.
    match = re.match(r'(\d{1,2}):(\d{2})', time_str)
    if match:
        minutes, seconds = map(int, match.groups())
        total_seconds = minutes * 60 + seconds
        if total_seconds == 0:
            return np.nan  
        return total_seconds

    # Do the same but for 0:00, 0:01, etc.
    match = re.match(r'(\d{1,2}):(\d{1,2})', time_str)
    if match:
        minutes, seconds = map(int, match.groups())
        total_seconds = minutes * 60 + seconds
        if total_seconds == 0:
            return np.nan 
        return total_seconds
    
    # If format doesn't match, return NaN
    return np.nan

# Apply the conversion function to the 'Time' column
races_df['Time_seconds'] = races_df['Time'].apply(time_to_seconds)

# print(races_df[['Time', 'Time_seconds']].head(10))
# races_df.info()

"""
Converting timelag to total seconds
"""

def timelag_to_seconds(timelag_str):
    # Check if the timelag_str contains missing values
    if pd.isna(timelag_str):
        return np.nan

    timelag_str = timelag_str.lstrip('+').strip()

    match = re.match(r'(\d{1,2}):(\d{2}):(\d{2})', timelag_str)
    if match:
        hours, minutes, seconds = map(int, match.groups())
        return hours * 3600 + minutes * 60 + seconds

    match = re.match(r'(\d{1,2}):(\d{2})', timelag_str)
    if match:
        minutes, seconds = map(int, match.groups())
        return minutes * 60 + seconds
    
    match = re.match(r'(\d+):(\d+)', timelag_str)
    if match:
        minutes, seconds = map(int, match.groups())
        return minutes * 60 + seconds

    return np.nan

races_df['Timelag_seconds'] = races_df['Timelag'].apply(timelag_to_seconds)

races_df['Timelag_seconds'] = races_df['Timelag_seconds'].replace(0.0, np.nan)

print(races_df[['Timelag', 'Timelag_seconds']].head(10))

"""
Converting distance into single numerical value
This means stripping 'km' from string and converting the remaining values into float64
"""

races_df['Length'] = races_df['Length'].str.replace(' km', '', regex=False)

# Convert to numeric and replace 0 with NaN
races_df['Length'] = pd.to_numeric(races_df['Length'], errors='coerce')  # Convert to numeric and handle errors

# Replace 0 values with NaN
races_df['Length'] = races_df['Length'].replace(0.0, np.nan)

# print(races_df['Length'])

"""
Splitting values from 'rdr' and putting the split values into separate columns
"""

# Function to convert the 'rdr' string to separate ranking columns
def extract_rankings(rdr_str):
    try:
        # Converting str to dict
        rankings = ast.literal_eval(rdr_str)
        
        # Extracting ranks, while also handling missing keys
        pcs_rnk = rankings.get('PCS Ranking', np.nan)
        uci_rnk = rankings.get('UCI World Ranking', np.nan)
        alltime_rnk = rankings.get('Specials | All Time Ranking', np.nan)
        
        return pd.Series([pcs_rnk, uci_rnk, alltime_rnk])
    except:
        return pd.Series([np.nan, np.nan, np.nan])

# Apply the function to the 'rdr' column
riders_df[['PCS_Rnk', 'UCI_Rnk', 'AllTime_Rnk']] = riders_df['rdr'].apply(extract_rankings)

# Entries should be turned into numeric values, where errors get turned into NaN
riders_df['PCS_Rnk'] = pd.to_numeric(riders_df['PCS_Rnk'], errors='coerce')
riders_df['UCI_Rnk'] = pd.to_numeric(riders_df['UCI_Rnk'], errors='coerce')
riders_df['AllTime_Rnk'] = pd.to_numeric(riders_df['AllTime_Rnk'], errors='coerce')

# print(riders_df[['fullname', 'PCS_Rnk', 'UCI_Rnk', 'AllTime_Rnk']])

"""
Convert stage types to binary with label encoding
"""

# races_df['Stage_Type_bin'] = races_df['Stage_Type'].map({'RR': 0, 'ITT': 1})

"""
Splitting values from 'pps' and putting those values into separate columns.
"""

def extract_points(pps_str):
    try:
        points = ast.literal_eval(pps_str)
        
        day_pnt = np.nan if points.get('One day races', '0') == '0' else points.get('One day races', np.nan)
        gc_pnt = np.nan if points.get('GC', '0') == '0' else points.get('GC', np.nan)
        tt_pnt = np.nan if points.get('Time trial', '0') == '0' else points.get('Time trial', np.nan)
        sprint_pnt = np.nan if points.get('Sprint', '0') == '0' else points.get('Sprint', np.nan)
        climb_pnt = np.nan if points.get('Climber', '0') == '0' else points.get('Climber', np.nan)
        
        return pd.Series([day_pnt, gc_pnt, tt_pnt, sprint_pnt, climb_pnt])
    except:
        return pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan])

riders_df[['Day_Pnt', 'GC_Pnt', 'TT_Pnt', 'Sprint_Pnt', 'Climb_Pnt']] = riders_df['pps'].apply(extract_points)

riders_df['Day_Pnt'] = pd.to_numeric(riders_df['Day_Pnt'], errors='coerce')
riders_df['GC_Pnt'] = pd.to_numeric(riders_df['GC_Pnt'], errors='coerce')
riders_df['TT_Pnt'] = pd.to_numeric(riders_df['TT_Pnt'], errors='coerce')
riders_df['Sprint_Pnt'] = pd.to_numeric(riders_df['Sprint_Pnt'], errors='coerce')
riders_df['Climb_Pnt'] = pd.to_numeric(riders_df['Climb_Pnt'], errors='coerce')


Amount of rows in races_df:  225918
Amount of rows in riders_df:  1042
Unique values for 'months':  [ 1  3  4  5  6  7  8  9 10] 

Unique values for 'years':  [2012 2014 2015 2017 2018 2020 2021]
  Timelag  Timelag_seconds
0   +0:00              NaN
1   +0:04              4.0
2   +0:06              6.0
3   +0:10             10.0
4   +0:10             10.0
5   +0:10             10.0
6   +0:10             10.0
7   +0:10             10.0
8   +0:10             10.0
9   +0:10             10.0


In [53]:
print(riders_df['fullname'])

0          BARDET Romain
1        DUMOULIN Samuel
2          GALLOPIN Tony
3          NAESEN Oliver
4          FRANK Mathias
              ...       
1037      TRONDSEN Trond
1038    VAN MELSEN Kévin
1039     BEULLENS Cédric
1040    DE WINTER Ludwig
1041      DELACROIX Théo
Name: fullname, Length: 1042, dtype: object


In [54]:
riders_df['fullname'] = riders_df['fullname'].str.replace(' ', '_') 
print(riders_df['fullname'])

0          BARDET_Romain
1        DUMOULIN_Samuel
2          GALLOPIN_Tony
3          NAESEN_Oliver
4          FRANK_Mathias
              ...       
1037      TRONDSEN_Trond
1038    VAN_MELSEN_Kévin
1039     BEULLENS_Cédric
1040    DE_WINTER_Ludwig
1041      DELACROIX_Théo
Name: fullname, Length: 1042, dtype: object


In [55]:
"""
Join the rider and race tables together, using the rider_id as an index
"""
print("Amount of rows in races_df post cleanup: ", races_df[races_df.columns[0]].count())
print("Amount of rows in riders_df post cleanup: ", riders_df[riders_df.columns[0]].count())

df = races_df.set_index('rider_id').join(riders_df.set_index('rider_id'), how = 'left')

print("Amount of rows in df: ", df[df.columns[0]].count())

"""
Dropping columns that are not needed for analysis
"""

# Note: fix the long list
df.drop(['Time', 'Timelag', 'rdr', 'pps', 'birthdate', 'rider_url', 'Race_url', 'Stage_url', 'Circuit', 'Race_Name', 'Stage_Name', 'Start', 'Finish', 'Category'], axis=1, inplace=True)

# Additional drops:
df = df.drop(columns=['id', 'Team', 'Date', 'Race_ID', 'Stage_Number', 'Team'])
# Replace all remaining zero values with NaN
df = df.replace(0, np.nan)

print(df.isna().sum())
print("Amount of rows in df after cleanup: ", df[df.columns[0]].count())
print("\ndone")

Amount of rows in races_df post cleanup:  80174
Amount of rows in riders_df post cleanup:  1042
Amount of rows in df:  80174
Rnk                    0
GC                 10059
BiB                    0
Rider                  0
Age                    0
UCI                77527
Pnt                69767
Stage_Type             0
Length              6495
Month                  0
Year                   0
Time_seconds       18435
Timelag_seconds    11212
fullname           34208
team               34208
country            34208
height             34260
weight             34266
PCS_Rnk            45842
UCI_Rnk            46804
AllTime_Rnk        53300
Day_Pnt            34216
GC_Pnt             34208
TT_Pnt             34631
Sprint_Pnt         34342
Climb_Pnt          34365
dtype: int64
Amount of rows in df after cleanup:  80174

done


### PI 7: Part 3


#### 1.1 Appropriate Machine Learning Models
In this section, the selection of appropriate machine learning models in the context of the current dataset will be discussed.

#### 1.2.1 Categorical and Regression Trees
For the first model, the use of Categorical and Regression Trees will be discussed. As stated in *part 2* of the assignment, CART is a decision tree, that classifies records based on the conditions in the *decision nodes*, where the final classification, or regression, is determined in the *leaf nodes*. See the code from <code>pi7-2.ipynb</code> for a more in-depth explanation of decision trees.

#### 1.2.2 Argumentation
A regression tree will be implemented due to the following reasons:
- Handling possible non-linear relationships.
- It does not require any normalization/standardization, making outcomes more interpretable.
- It can handle outliers and missing values well.

Arguments against implementing regression trees:
- The more complex a tree becomes, the more prone it becomes to overfitting. There are methods for handling scenarios where overfitting can become an issue, like *pruning*.
- Sensitive to hyperparameters.
- Unsuitable for datasets with a large number of classes (see the arguments above).

#### 1.2.3 Implementation
Below is am implementation of a regression tree on the current dataset.

In [56]:
df.isnull().sum()

Rnk                    0
GC                 10059
BiB                    0
Rider                  0
Age                    0
UCI                77527
Pnt                69767
Stage_Type             0
Length              6495
Month                  0
Year                   0
Time_seconds       18435
Timelag_seconds    11212
fullname           34208
team               34208
country            34208
height             34260
weight             34266
PCS_Rnk            45842
UCI_Rnk            46804
AllTime_Rnk        53300
Day_Pnt            34216
GC_Pnt             34208
TT_Pnt             34631
Sprint_Pnt         34342
Climb_Pnt          34365
dtype: int64

'Rnk' or final rank will be used as our predictor variable. This column still contains alot of numerical values for 'DNF', meaning this will need to get cleaned up. Removing non-numericals, like DNF, DNS and OTL, does not impact the amount of available data by a large amount, so we will result in dropping these entries. Since these entries will most likely not contain any useful information (since the rider never finished or fouled), these entries will be dropped.

In [57]:
df['Rnk'].isna().sum()

np.int64(0)

In [58]:
df['Rnk'] = pd.to_numeric(df['Rnk'], errors='coerce')

In [59]:
df['Rnk'].isna().sum()

np.int64(5219)

It might also be a good idea to distinguish between two different types of races: Road Race (RR) and Individual Time Trial (ITT). It's a possibility that some riders are more specialized in races and others in time trials, so to potentially improve accuracy, the dataframe will be split into two separate dataframes. For the machine learning models, RR will be used.

In [60]:
df_rr = df[df['Stage_Type'] == 'RR'].drop(columns='Stage_Type', axis=1)
df_itt = df[df['Stage_Type'] == 'ITT'].drop(columns='Stage_Type', axis=1)

Regression will be used to determine the outcome rank for riders. To do this, a new dataframe will be created to define the predictors of which we think are important for the outcome rank.

To make it simple for our regression tree, the outcome will be based on the personal ranking  and points predictors, height and weight and average time(lag).

In [61]:
dtr_drop = ['GC', 'BiB', 'UCI', 'Pnt', 'Rider', 'fullname', 'team', 'country', 'Length', 'Month', 'Year']
            #, 'Day_Pnt', 'GC_Pnt', 'TT_Pnt', 'Sprint_Pnt', 'Climb_Pnt']
df_ml = df_rr.drop(columns=dtr_drop)

df_ml.isnull().sum()
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72484 entries, 659ed585810c65fe22255a5e4a9b7838 to 0292146b9196ec7a98903cb50dae48cd
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rnk              67415 non-null  float64
 1   Age              72484 non-null  int64  
 2   Time_seconds     54176 non-null  float64
 3   Timelag_seconds  61451 non-null  float64
 4   height           41100 non-null  float64
 5   weight           41094 non-null  float64
 6   PCS_Rnk          30359 non-null  float64
 7   UCI_Rnk          29501 non-null  float64
 8   AllTime_Rnk      24218 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 5.5+ MB


In [62]:
df_ml = df_ml.dropna()
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12175 entries, a73d590113699f02caf57566c20a2ae7 to d6d7a1bb41a96c4bd23470d2c2eb58e8
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Rnk              12175 non-null  float64
 1   Age              12175 non-null  int64  
 2   Time_seconds     12175 non-null  float64
 3   Timelag_seconds  12175 non-null  float64
 4   height           12175 non-null  float64
 5   weight           12175 non-null  float64
 6   PCS_Rnk          12175 non-null  float64
 7   UCI_Rnk          12175 non-null  float64
 8   AllTime_Rnk      12175 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 951.2+ KB


In [63]:
df_ml['Rnk'].info

<bound method Series.info of rider_id
a73d590113699f02caf57566c20a2ae7     95.0
9be030c6ab77e67c18c3ddc46be4036c     97.0
f0e3e679300f9caf7c477c1d3614a33b    104.0
4485ac2bf252ae4395f1b850fa9e2c76    107.0
8b2ba2d2ad59c160774fc929f7f8a635    111.0
                                    ...  
605b2cfef3a235eae26095b0b58c9636     72.0
43280b81c89cf41e5e07c2acdf175be7     85.0
6c2f3519a714630b8a26b9e5211083c4     88.0
307a86e66444d89edf04251cfed9cca5     91.0
d6d7a1bb41a96c4bd23470d2c2eb58e8     93.0
Name: Rnk, Length: 12175, dtype: float64>

In [64]:
X = df_ml.iloc[:, 1:16] # adjust row val
y = df_ml.iloc[:, 0]

In [65]:
y

rider_id
a73d590113699f02caf57566c20a2ae7     95.0
9be030c6ab77e67c18c3ddc46be4036c     97.0
f0e3e679300f9caf7c477c1d3614a33b    104.0
4485ac2bf252ae4395f1b850fa9e2c76    107.0
8b2ba2d2ad59c160774fc929f7f8a635    111.0
                                    ...  
605b2cfef3a235eae26095b0b58c9636     72.0
43280b81c89cf41e5e07c2acdf175be7     85.0
6c2f3519a714630b8a26b9e5211083c4     88.0
307a86e66444d89edf04251cfed9cca5     91.0
d6d7a1bb41a96c4bd23470d2c2eb58e8     93.0
Name: Rnk, Length: 12175, dtype: float64

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17, test_size=0.2)

In [67]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

dtr = DecisionTreeRegressor().fit(X, y)

y_pred = dtr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean squared error: ", mse)

Mean squared error:  0.368788501026694


In [68]:
""" feature_names = df_ml.columns[1:16].tolist()

plt.figure(figsize=(25,12))
plot_tree(dtr, fontsize=8, feature_names=feature_names)
plt.show() """

' feature_names = df_ml.columns[1:16].tolist()\n\nplt.figure(figsize=(25,12))\nplot_tree(dtr, fontsize=8, feature_names=feature_names)\nplt.show() '

In [69]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

In [70]:
dtc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': None,
 'splitter': 'best'}

In [71]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [72]:
y_pred = dtc.predict(X_test)

In [73]:
from sklearn.metrics import confusion_matrix 

print(confusion_matrix(y_test, y_pred))

[[45  0  0 ...  0  0  0]
 [ 0  4  0 ...  0  0  0]
 [ 0  3  7 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [74]:
from sklearn.metrics import classification_report
accuracy = accuracy_score(y_test, y_pred)
print("accuracy: ", accuracy)

# Due to the large number of objects, overall accuracy score is used
# print(classification_report(y_test, y_pred, zero_division=0)) 

accuracy:  0.12361396303901437


In [75]:
features = pd.DataFrame(dtc.feature_importances_, index= X.columns)

features.head(15)

,0
Age,0.126208
Time_seconds,0.278891
Timelag_seconds,0.267737
height,0.059420
weight,0.061777
PCS_Rnk,0.067177
UCI_Rnk,0.065648
AllTime_Rnk,0.073142


In [76]:
dtc2 = DecisionTreeClassifier(criterion= 'entropy', ccp_alpha = 0.04)

dtc2.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.04, criterion='entropy')

In [77]:
y_pred2 = dtc2.predict(X_test)
print(confusion_matrix(y_test, y_pred2))

[[45  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [78]:
#print(classification_report(y_test, y_pred2, zero_division=0))
accuracy = accuracy_score(y_test, y_pred2)
print("accuracy: ", accuracy)

accuracy:  0.03162217659137577


In [79]:
features2 = (pd.DataFrame(dtc2.feature_importances_, index=X.columns))
features2.head(15)

,0
Age,0.000000
Time_seconds,0.884219
Timelag_seconds,0.115781
height,0.000000
weight,0.000000
PCS_Rnk,0.000000
UCI_Rnk,0.000000
AllTime_Rnk,0.000000


In [80]:
new_data = pd.DataFrame({'Age': [25], 'Length': [100], 'Month': [6], 'Year': [2022], 'Time_seconds': [3600], 'Timelag_seconds': [0], 'height': [180], 'weight': [70], 'PCS_Rnk': [10], 'UCI_Rnk': [20], 'AllTime_Rnk': [30], 'Day_Pnt': [40], 'GC_Pnt': [50], 'TT_Pnt': [60], 'Sprint_Pnt': [70]})

# Use the DTC model to make a prediction
prediction = dtc.predict(new_data)

# Print the prediction
print(prediction)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Day_Pnt
- GC_Pnt
- Length
- Month
- Sprint_Pnt
- ...


#### 1.2.4 Boosted Forest

#### 1.3 Gradient Boosting


In [249]:
y_test, y_train

(rider_id
 3397edbe10fdce2e38db0e4c9c253dad     51.0
 4b35e4a129ae080c4ccec66ea79e0be3    186.0
 98430c31927e1578acdcd383bab94dbb     67.0
 1dccfb04007c206da981d90978e2ee89     96.0
 e9b1a932c94dca07893dfede5eb12903    145.0
                                     ...  
 f1bd86cc6567c0dfa5721dce45d35e7e     56.0
 7d63fc210f1790a5b6a6d0fc3035e527     86.0
 d7e19a4f12a14a449d21900e818770ce     35.0
 ec94bc38e5f69ba0697bea8b47a9db10     14.0
 8ca4cfaf07a8e357bbcfbffbf2061b25     69.0
 Name: Rnk, Length: 2438, dtype: float64,
 rider_id
 1a1e24868c09fa2a54922a0fefdf4465    121.0
 c8220dbd975ae82181f98722000b662f     42.0
 5dbb77082695488e013574823562ec57     76.0
 7022fe00f5237209e9f58e8e7fff0446     50.0
 6b9154d3754796dd73e1ae2d5b27304b     75.0
                                     ...  
 22ef922b9767ce9d54e13c67bfd807ec     39.0
 64ce81af9f1e95f2417c7b761426b157     50.0
 db2d18b40afeb8ab98c08f7f6c19ddfc     32.0
 4485ac2bf252ae4395f1b850fa9e2c76     57.0
 783f549f704a01c53c60ffc3e40910d5  

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

#clf = HistGradientBoostingClassifier().fit(X, y)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.004101722723543888

In [251]:
accuracy_gb = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy_gb)

Accuracy:  0.030762920426579164


The number of regression trees is controlled by n_estimators. 
The size of each tree can be controlled either by setting the tree depth via max_depth or by setting the number of leaf nodes via max_leaf_nodes. The learning_rate is a hyper-parameter in the range (0.0, 1.0) that controls overfitting via shrinkage.

#### Source:

*Ensembles: Gradient boosting, random forests, bagging, voting, stacking.* (2022). Scikit-Learn. https://scikit-learn.org/stable/modules/ensemble.html#gradientboostingclassifier-and-gradientboostingregressor

‌

#### 1.4 Support Vector Regression (SVR)

#### 2.1 Metrics for Models

#### 3.1 Cross-validation

#### 4.1 Ideal Hyperparameters

#### 5.1 Conclusions

#### Bibliography